In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

In [3]:
import random 
import glob
import json
from tqdm import tqdm
import unicodedata

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import pytorch_lightning as pl

MODEL_NAME = 'bert-base-uncased'

In [4]:
from multi_classification import BertForSequenceClassificationMultiLabel

In [5]:
import pandas as pd
read_text_file = pd.read_csv("twitter-2015train-BD.csv", header=None)

In [6]:
X_train = read_text_file[3]
y_train = read_text_file[2]

In [7]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
bert_scml = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=3
) 

In [8]:
list = []
label = []
for i , j in zip(X_train, y_train):
    list.append(i)
    label.append(j)

In [9]:
labels = []
for emotion in label:
    if emotion == 'negative':
        labels.append([0, 0, 1])
    elif emotion == ' neutral':
        labels.append([0, 1, 0])
    else:
        labels.append([1, 0, 0])

In [10]:
max_length = 128
dataset_for_loader = []

for i in range(len(list)):
    text = list[i]
    encoding = tokenizer(
      text, 
      max_length=max_length,
      padding='max_length',
      truncation=True
    )
    encoding['labels'] = labels[i]
    encoding = {k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
    

random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]


In [11]:
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(
    dataset_val, batch_size=256
)
dataloader_test = DataLoader(
    dataset_test, batch_size=256
)

In [13]:
from pytorch_lightning_multi import BertForSequenceClassificationMultiLabel_pl

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

#学習方法指定
trainer = pl.Trainer(
    max_epochs=5,
    callbacks=[checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, num_labels=3, lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type                                    | Params
----------------------------------------------------------------------
0 | bert_scml | BertForSequenceClassificationMultiLabel | 109 M 
----------------------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.938   Total estimated model params size (MB)
/Users/yuto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/Users/yuto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/yuto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.8585858585858586}
--------------------------------------------------------------------------------
Accuracy: 0.86
